# LLM

## Global

In [115]:
import torch
import random
import numpy as np

seed = 1337
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

device = "cpu"

## Dataset

### Load Dataset

In [116]:
with open("input.txt") as f:
    txt = f.read()
chars = sorted(list(set(txt)))
vocab_len = len(chars)
print("".join(chars))
print(vocab_len)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### Tokenisation / Detokenisation

In [117]:
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


### Dataloader

In [118]:
import torch
data = torch.tensor(encode(txt))
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [119]:
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target  = yb[b, t]
        print(f"Context: {context.tolist()}, Target: {target}")

Context: [53], Target: 59
Context: [53, 59], Target: 6
Context: [53, 59, 6], Target: 1
Context: [53, 59, 6, 1], Target: 58
Context: [53, 59, 6, 1, 58], Target: 56
Context: [53, 59, 6, 1, 58, 56], Target: 47
Context: [53, 59, 6, 1, 58, 56, 47], Target: 40
Context: [53, 59, 6, 1, 58, 56, 47, 40], Target: 59
Context: [49], Target: 43
Context: [49, 43], Target: 43
Context: [49, 43, 43], Target: 54
Context: [49, 43, 43, 54], Target: 1
Context: [49, 43, 43, 54, 1], Target: 47
Context: [49, 43, 43, 54, 1, 47], Target: 58
Context: [49, 43, 43, 54, 1, 47, 58], Target: 1
Context: [49, 43, 43, 54, 1, 47, 58, 1], Target: 58
Context: [13], Target: 52
Context: [13, 52], Target: 45
Context: [13, 52, 45], Target: 43
Context: [13, 52, 45, 43], Target: 50
Context: [13, 52, 45, 43, 50], Target: 53
Context: [13, 52, 45, 43, 50, 53], Target: 8
Context: [13, 52, 45, 43, 50, 53, 8], Target: 0
Context: [13, 52, 45, 43, 50, 53, 8, 0], Target: 26
Context: [1], Target: 39
Context: [1, 39], Target: 1
Context: [1,

In [120]:
xb.shape, yb.shape

(torch.Size([4, 8]), torch.Size([4, 8]))

## Bigram Model

### Define the Model

In [190]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLM(nn.Module):
    def __init__(self, vocab_len):
        super().__init__()
        self.token_emb_table = nn.Embedding(vocab_len, vocab_len)
    
    def forward(self, idx, targets=None):
        logits = self.token_emb_table(idx)
        if not targets is None:
            B, T, C = logits.shape
            logits  = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
model = BigramLM(vocab_len)
out, loss = model(xb, yb)
print(out.shape, loss)

torch.Size([32, 65]) tensor(4.6810, grad_fn=<NllLossBackward0>)


In [191]:
def gen():
    print(
            decode(
            model.generate(
                torch.zeros(
                    (1, 1), dtype=torch.long
                ),
                max_new_tokens=100
            )[0].tolist()
        )
    )
gen()


fBLd
zYAQ;i'U3Q,rV jLE:3QrrYsxP.Ii?,U$b jPg.3kdFTKfHcMCwzqJwUljiVCTdF.Yd
XB-ETtwDfR!P&d,laWtwx'Dw&Mb


### Train

In [155]:
optim = torch.optim.AdamW(model.parameters(), lr=1e-3)
bs = 32
for epoch in range(5000):
    xb, yb = get_batch("train")
    logits, loss = model(xb, yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()

print(loss.item())

2.4181735515594482


In [176]:
gen()


Ante theesth s ar d tsiss Wa bandig t?
d dad he fth-frit Flan pppogh's I ingr coJn; fqPKre WHI tino 


## Math Trick of Self-Attention

In [125]:
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape
torch.Size([4, 8, 2])

torch.Size([4, 8, 2])

In [126]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [127]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)

True

In [128]:
tril  = torch.tril(torch.ones(T, T))
wei   = torch.zeros((T, T))
wei   = wei.masked_fill(tril == 0, float("-inf"))
wei   = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [129]:
wei.shape, xbow3.shape

(torch.Size([8, 8]), torch.Size([4, 8, 2]))

In [130]:
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print(a)
print(b)
print(c)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[9., 0.],
        [4., 7.],
        [0., 4.]])
tensor([[13., 11.],
        [13., 11.],
        [13., 11.]])


In [131]:
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[1., 8.],
        [7., 5.],
        [8., 2.]])
tensor([[1.0000, 8.0000],
        [4.0000, 6.5000],
        [5.3333, 5.0000]])


## Bigram Model + Attention Head

### Define the Model

In [230]:
import torch.nn as nn
from torch.nn import functional as F

class Head(nn.Module):
    def __init__(self, head_size, n_embed, block_size):
        super().__init__()
        self.k = nn.Linear(n_embed, head_size, bias=False)
        self.q = nn.Linear(n_embed, head_size, bias=False)
        self.v = nn.Linear(n_embed, head_size, bias=False)

        self.register_buffer("tril",
                             torch.tril(torch.ones(block_size, block_size)))
    def forward(self, x):
        B, T, C = x.shape
        k = self.k(x)
        q = self.k(x)
        w = q @ k.transpose(-2, -1) * C ** -0.5
        w = w.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        w = F.softmax(w, dim=-1)
        v = self.v(x)
        o = w @ v
        return o
    
class BigramAttn(nn.Module):
    def __init__(self, vocab_len, n_embed):
        super().__init__()
        self.token_emb_table    = nn.Embedding(vocab_len, n_embed)
        self.position_emb_table = nn.Embedding(block_size, n_embed)
        self.sa_head = Head(n_embed, n_embed, block_size)
        self.lm_head = nn.Linear(n_embed, vocab_len)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_embed = self.token_emb_table(idx)
        pos_embed = self.position_emb_table(
            torch.arange(T, device=device))
        x = tok_embed + pos_embed
        x = self.sa_head(x)
        logits = self.lm_head(x)

        if not targets is None:
            B, T, C = logits.shape
            logits  = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramAttn(vocab_len, 128)
out, loss = model(xb, yb)
print(out.shape, loss)


torch.Size([32, 65]) tensor(4.3461, grad_fn=<NllLossBackward0>)


### Train

In [234]:
optim = torch.optim.AdamW(model.parameters(), lr=1e-3)
bs = 32
for epoch in range(1000):
    xb, yb = get_batch("train")
    logits, loss = model(xb, yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()

print(loss.item())

2.0144097805023193


In [246]:
gen()


Whiend hir kir herak:
Seses m y m fetellord for

Ty ded ous t ors!
Fout, IOmp, sy Wharis t maled; o 
